In [5]:
import numpy as np
import random
import pandas as pd

In [42]:
class GenAlg:

    def __init__(self, population_size, k_crossover, mutation_prob, rates, sigma):
        self.population_size = population_size
        self.k = k_crossover
        self.mutation_prob = mutation_prob
        self.number_of_stocks = len(rates)

        self.rates = rates.copy()
        self.sigma = sigma.copy()

        self.random_pop()
    


    def random_pop(self):
        self.population = np.random.rand(self.population_size,self.number_of_stocks)
        for  i in range(self.population.shape[0]):
            self.population[i,] = self.normalize(self.population[i,])
    
    def normalize(self,vector):
        sum_of_vector = sum(vector)
        for i in range(len(vector)):
            vector[i] =  vector[i]/sum_of_vector
        return vector
    
    def evaluate(self):
        best_fitness = None
        best = None
        for vector in self.population:
            f = self.fitness(vector)
            if best is None or f > best_fitness:
                best = vector
                best_fitness = f
        return best_fitness,best

    def fitness(self,vector):
        rate = np.sum(vector * self.rates)
        volatility = np.sqrt(np.dot(vector.T,np.dot(self.sigma,vector)))

        return rate/volatility

    def get_lucky_one(self):
        random_num = random.randrange(len(self.population))
        return self.population[random_num,:]
    
    def select_parents(self,lucky_one):
        # if lucky_one exists then remove him
        if lucky_one is not None:
            pop = np.ones(shape=(self.population_size-1,self.number_of_stocks))
            i = 0
            for vec in self.population:
                if not (vec == lucky_one).all():
                    pop[i,:] = vec
                    i+=1
        else:
            pop = self.population.copy()
        
        new_pop = np.array(sorted(pop,key=self.fitness,reverse=True))

        return new_pop[:len(new_pop)//2,:]

    def create_children(self,parents):
        
        children = np.zeros(shape=(self.population_size,self.number_of_stocks))
        np.random.shuffle(parents)
        first_p, second_p = np.split(parents,2)
        for i in range(len(parents)//2):
            parent1 = first_p.pop(0)
            parent2 = second_p.pop(0)

            child1, child2 = self.k_crossover(parent1,parent2)

            child1 = self.mutation(child1)
            child2 = self.mutation(child2)

            child1 = self.normalize(child1)
            child2 = self.normalize(child2)

            children[i,:] = child1
            children[i+len(parents)//2,:] = child2

        return children 
    
    def k_crossover(self,x,y):

        slices = set()
        while len(slices) < self.k-1:
            slice_index = random.choice([i for i in range(1,len(x)) if i not in slices])
            if slice_index not in slices:
                slices.add(slice_index)

        slices = sorted(list(slices))
        slices.append(self.number_of_stocks)

        new_x = np.zeros(self.number_of_stocks)
        new_y = np.zeros(self.number_of_stocks)

        first = 0
        switch = True
        for second in slices:
            if switch:
                new_x[first:second] = x[first:second]
                new_y[first:second] = y[first:second]
            else:
                new_x[first:second] = y[first:second]
                new_y[first:second] = x[first:second]
            first = second
            switch = not switch

        return new_x,new_y

    def mutation(self,child):
        for i in range(child):
            if random.random() < self.mutation_prob:
                child[i] += random.gauss(0,0.5)
        return child

    def solve(self,max_generations, goal):
        
        best_fit, best = self.evaluate()
        
        for _ in range(max_generations):
            if best_fit > goal:
                break

            lucky_one = None
            if len(self.population)%2 == 1:
                lucky_one = self.get_lucky_one()

            parents = self.select_parents(lucky_one)

            children = self.create_children(parents)

            if lucky_one is not None:
                lucky_one = lucky_one.reshape((1,len(lucky_one)))
            self.population = np.concatenate((lucky_one,parents,children),axis=0)

            best_fit, best = self.evaluate()
        return best,best_fit

In [44]:
wanted_stocks = ['GOOG', 'SPG', 'GOOGL', 'MSFT', 'GD', 'ACN', 'COP', 'F', 'BAC', 'GS',
                'NVDA', 'AIG', 'MS', 'WFC', 'ORCL', 'XOM', 'TGT', 'LOW', 'EXC', 'COST',
                'AXP', 'BK', 'JPM', 'COF', 'CSCO', 'DHR', 'UNH', 'CVS', 'LLY', 'CVX',
                'MET', 'AMT', 'CRM', 'BLK', 'RTX', 'MCD', 'TMO', 'LIN', 'ADBE', 'EMR',
                'USB', 'UPS', 'TSLA', 'PFE', 'PM']
stocks = pd.read_csv('data/sap100_data_08112021.csv',index_col=0).loc[:,wanted_stocks]
returns = stocks/stocks.shift(1)-1
rates = returns.mean() * 252
sigma = returns.cov() * 252
g = GenAlg(10, 5, 0.01, rates,sigma)

best,best_fit = g.solve(10,1)
print(best,best_fit)
# print(g.population[0,:],'\n',g.population[1,:])

# g.k_crossover(g.population[0,:],g.population[1,:])

# test_pop = np.zeros(shape=(5,45))
# for i in range(5):
#     test_pop[i,i] = 1
# print(test_pop)
# for i in range(5):
#     print(g.fitness(test_pop[i,:]))
# g.population = test_pop
# v = g.get_lucky_one()
# print('lucky one: ',v)
# print('paretns')
# print(g.select_parents(v))

[0.01114155 0.02112081 0.0026389  0.04158689 0.02911615 0.00810653
 0.00953936 0.01291289 0.03460458 0.0172124  0.02812051 0.00615008
 0.03882499 0.01603352 0.03222546 0.02002055 0.01096212 0.00879385
 0.02504149 0.02297195 0.04096784 0.04078883 0.00404298 0.00336519
 0.02037814 0.0395612  0.041175   0.03722034 0.01283837 0.02809674
 0.0232359  0.03603028 0.04134022 0.00582411 0.01888116 0.03858949
 0.00809903 0.03437509 0.03463708 0.00438613 0.02915961 0.01188739
 0.00890677 0.02284441 0.01624411] 3.7473738510323478


In [47]:
a = [0.01114155, 0.02112081, 0.0026389 , 0.04158689 ,0.02911615 ,0.00810653,
 0.00953936, 0.01291289, 0.03460458, 0.0172124,  0.02812051, 0.00615008,
 0.03882499, 0.01603352 ,0.03222546, 0.02002055, 0.01096212, 0.00879385,
 0.02504149, 0.02297195, 0.04096784, 0.04078883, 0.00404298 ,0.00336519,
 0.02037814, 0.0395612 , 0.041175,   0.03722034, 0.01283837, 0.02809674,
 0.0232359 , 0.03603028 ,0.04134022, 0.00582411, 0.01888116, 0.03858949,
 0.00809903, 0.03437509, 0.03463708 ,0.00438613 ,0.02915961, 0.01188739,
 0.00890677, 0.02284441, 0.01624411]
max(a)

0.04158689

In [50]:
w = np.array(a)
r = np.sum(w*rates)
print(r)
v = np.sqrt(np.dot(w.T,np.dot(sigma,w)))
print(v)
print(r/v)

0.4701581146929166
0.12546335975815912
3.747373859580875
